# Etapa do Pré-Processamento
Etapa do pré-processamento dos dados para depois, alimentarmos esses dados ao modelo.

Nesta etapa realizamos:
* Transformação de todas as palavras para minúsculas;
* Remoção de pontuações;
* Remoção de stopwords;
* Aplicamos o stemming;
* Divisão entre treino, validação e teste;
* Tokenização;
* Padding.

> **Nota**: **Artigo no Medium** da etapa do `pré-processemanto` desse sistema em português: [Análise de Sentimentos Sobre os Quadrinhos da Marvel (Parte 1) - Ingestão, EDA e Pré-processamento](https://medium.com/@guineves.py/c5a0e35bb586).

## Table of Contents
* [Pacotes](#1)
* [Pré-processamento](#2)
    * [Pré-processamento da RNN](#2.1)
        * [Lowercasing, Stopwords, Stemming e Pontuações](#2.1.1)
        * [Tokenização](#2.1.2)
    * [Pré-processamento do Transformer](#2.2)

<a name="1"></a>
## Packages (Pacotes)
Pacotes que foram utilizados no sistema:
* [pandas](https://pandas.pydata.org/): é o principal pacote para manipulação de dados;
* [numpy](www.numpy.org): é o principal pacote para computação científica;
* [re](https://docs.python.org/3/library/re.html): fornece operações de correspondência de expressões regulares semelhantes às encontradas em Perl;
* [string](https://docs.python.org/pt-br/3.13/library/string.html): para operações comuns de strings;
* [nltk](https://www.nltk.org/): NLTK é uma plataforma líder para a construção de programas Python para trabalhar com dados de linguagem humana;
* [tensorflow](https://www.tensorflow.org/): framework que facilita a criação de modelos de machine learning que podem ser executados em qualquer ambiente;
* [scikit-learn](https://scikit-learn.org/stable/): biblioteca open-source de machine learning;
* [pickle](https://docs.python.org/3/library/pickle.html): implementa protocolos binários para serializar e desserializar uma estrutura de objeto Python;
* [transformers](https://huggingface.co/docs/transformers/index): fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração;
* [datasets](https://huggingface.co/docs/datasets/index): é uma biblioteca para acessar e compartilhar facilmente datasets para tarefas de áudio, visão computacional e processamento de linguagem natural (NLP);
* [os](https://docs.python.org/3/library/os.html): módulo integrado, fornece uma maneira portátil de usar funcionalidades dependentes do sistema operacional;
* [sys](https://docs.python.org/3/library/sys.html): fornece acesso a algumas variáveis usadas ou mantidas pelo interpretador e a funções que interagem fortemente com o interpretador;
* [src](../src/): pacote com todos os códigos de todas as funções utilitárias criadas para esse sistema. Localizado dentro do diretório `../src/`.

In [107]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedShuffleSplit
import pickle
from transformers import DistilBertTokenizer
from datasets import Dataset

import os
import sys
PROJECT_ROOT = os.path.abspath( # Obtendo a versão absoluta normalizada do path raíz do projeto
    os.path.join( # Concatenando os paths
        os.getcwd(), # Obtendo o path do diretório dos notebooks
        os.pardir # Obtendo a string constante usada pelo OS para fazer referência ao diretório pai
    )
)
# Adicionando o path à lista de strings que especifica o path de pesquisa para os módulos
sys.path.append(PROJECT_ROOT)
from src.preprocessing import *

> **Nota**: os códigos para as funções utilitárias utilizadas nesse sistema estão no script `preprocessing.py` dentro do diretório `../src/`.

<a name="2"></a>
## Pré-processamento
Vamos criar 2 modelos para esse projeto. Portanto, faremos 2 pré-processamentos diferentes, um para cada modelo:
1. O primeiro será utilizando uma RNN com uma layer LSTM bidirecional criadas e treinada do zero.
2. O segundo modelo será a aplicação de fine-tuning em um modelo com arquitetura transformers pré-treinado.

Lendo o dataset que será pré-processado e projetando os seus primeiros 5 exemplos.

In [50]:
comics_data = pd.read_csv('../data/raw/comics_corpus.csv')
comics_data.head()

,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...,non-action
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...,non-action
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...,non-action


Podemos ver algumas frases duplicadas na feature `description`. Mas se explorarmos mais de perto, podemos ver que o que não faz elas se tornarem 100% similares, são as pontuações. Portanto, vamos tratar as pontuações para eliminar os exemplos duplicados.

In [52]:
print(f'Algum exemplo:\n{comics_data["description"][1]}\n')
print(f'Exemplo duplicado:\n{comics_data["description"][2]}')

Algum exemplo:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn’t Hel’s only ruler—and now she’s upset the cosmic balance. There will be a price to pay…and Karnilla intends to ensure the Valkyries pay it.

Exemplo duplicado:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn?t Hel?s only ruler?and now she?s upset the cosmic balance. There will be a price to pay?and Karnilla intends to ensure the Valkyries pay it.


<a name="2.1"></a>
### Pré-processamento da RNN
Para um classificador de sentimentos, primeiro pré-processamos os dados brutos, depois, tokenizamos o nosso train set e extraímos as features úteis para treinarmos o nosso modelo e fazermos nossas previsões.

Plotando as stopwords em ingês e as pontuações que serão removidas.

In [57]:
stopwords_en = stopwords.words('english')
punct = string.punctuation
print(f'Stopwords:\n{stopwords_en}\n\nPontuações:\n{punct}')

Stopwords:
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so'

<a name="2.1.1"></a>
#### Lowercasing, Stopwords, Stemming e Pontuações
Após o pré-processamento inicial dos dados, acabamos apenas com as palavras que contêm todas as informações relevantes sobre o texto. Pré-processamentos iniciais antes da divisão e tokenização:
* `Lowercasing`: para reduzir nosso vocabulário sem perder informações valiosas, teremos que colocar cada uma de nossas palavras em lowercase. Portanto, a palavra CHILDREN, Children e children, serão tratadas como sendo exatamente a mesma palavra children.
* `Caracteres especiais`: como símbolos matemáticos, símbolos monetários, sinais de seção e parágrafo, sinais de marcação inline e assim por diante. Geralmente é seguro excluí-los.
* `Stopwords e pontuações`: removemos todas as palavras que não acrescentam significado significativo aos textos, também conhecidas como stopwords e sinais de pontuação (punctuation marks). Após eliminadas, o significado geral da frase pode ser inferido sem nenhum esforço.
* `Stemming`: é simplesmente transformar qualquer palavra em sua base stem (raiz base), que podemos definir como o set de caracteres usados para construir a palavra e seus derivados. A palavra works por exemplo, seu stem é `work`, porque, adicionando a letra "s", forma a palavra works, adicionando o sufixo "e", forma a palavra worke, e adicionando o sufixo "ing", forma a palavra working.
    * Depois de executar a stemming no nosso corpus, as palavras works, worke e working, serão reduzidas para a stem `work`. Portanto, nosso vocabulário será reduzido significativamente ao realizar esse processo para cada palavra do corpus.

Pré-processando os dados, contando os exemplos inicialmente pré-processados e duplicados, e plotando os primeiros 5 primeiros exemplos do dataset inicialmente pré-processado.

In [61]:
comics_data_pre = comics_data.copy()
comics_data_pre['description'] = comics_data_pre['description'].map(rnn_preprocess)
print(f'Número de exemplos duplicados: {comics_data_pre["description"].duplicated().sum()}')
comics_data_pre.head()

Número de exemplos duplicados: 790


,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),shadow kirisaki mountain secret histori come l...,non-action
1,93339,The Mighty Valkyries (2021) #3,children afterlif kraven hunter stalk jane fos...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),children afterlif kraven hunter stalk jane fos...,action
3,93350,X-Corp (2021) #2,shark water x corp shock debut got fenc mend h...,non-action
4,94896,X-Corp (2021) #2 (Variant),shark water x corp shock debut got fenc mend h...,non-action


Agora podemos remover os exemplos duplicados, porque estão inicialmente pré-processados e sem pontuações.

Plotando o número de duplicatas após a removação das duplicatas e plotando os primeiros 5 exemplos do dataset pré-processado e sem exemplos duplicados.

In [64]:
comics_data_pre = comics_data_pre.drop_duplicates('description')
print(f'Número de exemplos duplicados: {comics_data_pre["description"].duplicated().sum()}')
comics_data_pre.head()

Número de exemplos duplicados: 0


,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),shadow kirisaki mountain secret histori come l...,non-action
1,93339,The Mighty Valkyries (2021) #3,children afterlif kraven hunter stalk jane fos...,action
3,93350,X-Corp (2021) #2,shark water x corp shock debut got fenc mend h...,non-action
5,93645,Heroes Reborn: Weapon X & Final Flight (2021) #1,best world without aveng squadron suprem prote...,non-action
6,93052,Heroes Reborn (2021) #6,eon fabl daughter utopia isl known power princ...,non-action


Criando um dataset com apenas as features que serão usadas no modelo, transformando o target label $y$ em binário e plotando os primeiros 5 exemplos do dataset.

In [67]:
# Definindo o novo dataset
comics_corpus = comics_data_pre[['description', 'y']].copy()
# Transformando o target label y em binário
comics_corpus['y'] = comics_corpus['y'].map(lambda x: 1 if x == 'action' else 0)
comics_corpus.head()

,description,y
0,shadow kirisaki mountain secret histori come l...,0
1,children afterlif kraven hunter stalk jane fos...,1
3,shark water x corp shock debut got fenc mend h...,0
5,best world without aveng squadron suprem prote...,0
6,eon fabl daughter utopia isl known power princ...,0


Dividindo o dataset entre os subsets de treinamento, de validação e de teste. Usamos o split método `stratified sampling (amostragem estratificada)` para tentarmos manter a mesma proporção dos labels na divisão entre cada subset, porque temos um leve desbalanceamento de classes e não queremos que isso afete o nosso modelo.

Dividindo o dataset e plotando a dimensão de cada um deles.

In [71]:
# Dividindo entre treinamento e o subset da validação e teste
split_train = StratifiedShuffleSplit(n_splits=1, test_size=.4, random_state=42)
for train_index, subset_index in split_train.split(comics_corpus, comics_corpus['y']):
    train_corpus, subset_corpus = comics_corpus.iloc[train_index, :].copy(), comics_corpus.iloc[subset_index, :].copy()

# Dividindo entre validação e teste
split_test = StratifiedShuffleSplit(n_splits=1, test_size=.5, random_state=42)
for val_index, test_index in split_test.split(subset_corpus, subset_corpus['y']):
    val_corpus, test_corpus = subset_corpus.iloc[val_index, :].copy(), subset_corpus.iloc[test_index, :].copy()

print(f'Shape do subset de treinamento: {train_corpus.shape}\nShape do subset de valiação: {val_corpus.shape}\nShape do subset de teste: {test_corpus.shape}')

Shape do subset de treinamento: (9682, 2)
Shape do subset de valiação: (3227, 2)
Shape do subset de teste: (3228, 2)


Definindo as variáveis globais `VOCAB_SIZE` e `MAX_LEN` para tokenizar o training set.

In [74]:
VOCAB_SIZE = 1000
MAX_LEN = max([len(sentence.split()) for sentence in train_corpus['description']])
print(f'Tamanho da maior sequência limpa: {MAX_LEN}')

Tamanho da maior sequência limpa: 166


<a name="2.1.2"></a>
#### Tokenização
Nessa etapa, codificamos o nosso corpus do training set em sua representação vetorial, ou seja, primeiro precisamos criar um vocabulário $V$ que nos permite codificar qualquer texto como um vetor de inteiros, por exemplo. Onde o nosso vocabulário $V$ será um vetor de palavras exclusivas do nosso vetor de textos, onde percorremos cada palavra de cada texto e salvamos no vocabulário todas as novas palavras que aparecerem em nossa pesquisa. Então, mapeamos, ou seja, substituímos cada palavra encontrada no training set pelo seu índice no vocabulário.
| Token | Índice |
| :---: | :----: |
| afterlif | 1 |
| $\vdots$ | $\vdots$ |
| balanc | 615 |
| $\vdots$ | $\vdots$ |
| cosmic | 621 |
| $\vdots$ | $\vdots$ |
| work | VOCAB_SIZE |

A RNN pode receber como input uma representação vetorial de frases tokenizadas e, então, aplicamos uma embedding layer que transformará nossa representação tokenizada de inteiros em valores que representam a semântica dessa palavra numéricamente. Isso trata um dos problemas, a ordem das palavras, ordem alfabética, neste exemplo, não faz muito sentido do ponto de vista semântico. Por exemplo, não há razão para que o 'cosmic' receba um número maior do que o 'afterlif'.
> Falarei mais sobre embeddings na criação do modelo.

Essa RNN nos permitirá prever sentimentos em frases complexas, que não conseguiríamos classificar corretamente usando métodos mais simples, como Naive Bayes, porque eles perdem informações importantes.

A nossa representação $X$ será um vetor de números inteiros, ou seja, o índice de cada token do nosso vocabulário. Depois de obtermos todas as representações vetoriais das nossas frases, precisamos identificar o tamanho máximo do vetor e preencher cada vetor com 0 para corresponder a essa tamanho, esse processo é chamado de `padding` e garante que todos os nossos vetores tenham o mesmo tamanho, mesmo que nossas frases não tenham.

Para a criação do vocabulário, definimos quais palavras pertencem ao vocabulário. Isso significa que criamos uma lista com as palavras que vamos usar nas nossas representações. Uma forma de criar esse vocabulário é olhar para o nosso training set, e encontrar as `VOCAB_SIZE` palavras com mais ocorrência, por exemplo, ou utilizamos dicionários já criados que nos digam as `VOCAB_SIZE` palavras mais comumente usadas na língua da nossa tarefa.

Em algumas tarefas como speech recognition, ou question aswering, encontraremos e geraremos somente palavras a partir de um set fixo de palavras, por exemplo, um chatbot só pode responder em sets limitados de perguntas. Essa lista fixa de palavras também é chamada de `closed vocabulary`. No entanto, usar um set fixo de palavras nem sempre é suficiente para a tarefa. Muitas vezes, precisamos lidar com palavras que nunca vimos antes, o que resulta em um `open vocabulary`. **Open vocabulary** significa simplesmente que podemos encontrar palavras out of vocabulary (fora do vocabulário), como o nome de uma nova cidade no training set.

Se treinarmos uma rede neural em um corpus de textos com base nesse nosso vocabulário, quando quisermos fazer inferência com o modelo treinado, precisaremos codificar o texto que desejamos inferir com o mesmo vocabulário. Caso contrário, não fará sentido, porque as palavras seriam mapeadas para números diferentes, tokens diferentes. Qualquer palavra no corpus de treino que não esteja no vocabulário será substituída por `<UNK>`. As unknown words também são chamadas de `Out of vocabulary (OOV)`. Uma forma de lidar com palavras OOV é modelá-las com uma palavra especial, **\<UNK>**. Para fazermos isso, substituímos todas as palavras OOV por \<UNK>, o special token, `<UNK>`. A proporção de unknown words no test set é chamada de `OOV rate`.

No final, aplicamos o `padding`, adicionamos 0 ao final de cada sequence e fazemos com que todas tenham o mesmo comprimento. Isso é chamado de `post-padding (pós-padding)`, porque os tokens de padding estão no final das sequências.

O tensorflow e o Keras nos oferecem várias maneiras de tokenizar palavras. Uma delas é a layer que estou usando nesse sistema, [`tensorflow.keras.layers.TextVectorization()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization).
* `TextVectorization()`: gerará o vocabulário e criará vetores a partir das frases. Ela retira pontuações, ou seja, ela irá gerenciar os tokens, transformando as frases em uma lista de inteiros (os índices de cada token no vocabulário) e etc. 
* `adapt()`: método da layer TextVectorization(), que pega os dados e gera um vocabulário a partir das palavras encontradas nessas frases.

Treinando o tokenizador no training set com o `VOCAB_SIZE` e `MAX_LEN` definidos anteriormente e projetando o tamanho do vocabulário.

In [80]:
sentence_vec = rnn_tokenizer(train_corpus['description'], max_tokens=VOCAB_SIZE, max_len=MAX_LEN)
print(f'Tamanho do vocabulário: {sentence_vec.vocabulary_size()}')

Tamanho do vocabulário: 1000


Aplicando o tokenizador treinado em cada subset e plotando as suas dimensões e o primeiro exemplo tokenizado do training set.

In [83]:
train_tokenized = sentence_vec(train_corpus['description'])
val_tokenized = sentence_vec(val_corpus['description'])
test_tokenized = sentence_vec(test_corpus['description'])
print(f'Dimensão das sequências de treino tokenizadas e com padding: {train_tokenized.shape}\n\nPrimeira sequência de treino com padding:\n{train_tokenized[0]}\n')
print(f'Dimensão das sequências de validação tokenizadas e com padding: {val_tokenized.shape}')
print(f'Dimensão das sequências de teste tokenizadas e com padding: {test_tokenized.shape}')

Dimensão das sequências de treino tokenizadas e com padding: (9682, 166)

Primeira sequência de treino com padding:
[202 101  80  45 332 101  80 804 323 767   1   1 815 649 499 795 303  92
 409 624   3   1   1   1   1 413   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]

Dimensão das sequências de validação tokenizadas e com padding: (3227, 166)
Dimensão das sequências de teste tokenizadas e com padding: (3228, 166)


Carregando o modelo de tokenização treinado no diretório `../models/` para usarmos posteriormente. Salvamos os hiperparâmetros que foram utilizados no treinamento e o vocabulário gerado.

In [34]:
pickle.dump(
    {'config': sentence_vec.get_config(), 'vocabulary': sentence_vec.get_vocabulary()},
    open('../models/vectorizer.pkl', 'wb')
)

Transformando os labels $y$ em um vetor de coluna, concatenando cada corpus tokenizado com os labels $y$ correspondentes e plotando a dimensão de cada subset.

In [87]:
# Transformando os labels y em um vetor de coluna
labels_train = train_corpus[['y']].copy()
labels_val = val_corpus[['y']].copy()
labels_test = test_corpus[['y']].copy()

# Concatenando o corpus de cada subset e os labels correspondentes
train_tokens = np.concatenate([train_tokenized, labels_train], axis=1)
val_tokens = np.concatenate([val_tokenized, labels_val], axis=1)
test_tokens = np.concatenate([test_tokenized, labels_test], axis=1)
print(f'Dimensão do subset de treino pré-processado: {train_tokens.shape}')
print(f'Dimensão do subset de validação pré-processado: {val_tokens.shape}')
print(f'Dimensão do subset de teste pré-processado: {test_tokens.shape}')

Dimensão do subset de treino pré-processado: (9682, 167)
Dimensão do subset de validação pré-processado: (3227, 167)
Dimensão do subset de teste pré-processado: (3228, 167)


Carregando cada dataset pré-processado no diretório `../data/preprocessed/`.

In [47]:
# Carregando no disco o dataset com pré-processamento inicial
comics_corpus.to_csv('../data/preprocessed/comics_corpus.csv', index=False)
# Carregando no disco os datasets tokenizados
np.save('../data/preprocessed/train_tokens.npy', train_tokens)
np.save('../data/preprocessed/validation_tokens.npy', val_tokens)
np.save('../data/preprocessed/test_tokens.npy', test_tokens)

<a name="2.2"></a>
### Pré-processamento do Transformers
Para o pré-processamento do transformers, vamos utilizar um tokenizador pré-treinado do checkpoint do `DistilBERT` para aplicar a tokenização e o padding.

DistilBERT é um modelo Transformers pequeno, rápido, barato e leve treinado pela destilação do modelo base BERT (Bidirectional Encoder Representation from Transformers). Ele tem 40% menos parâmetros que o bert-base-uncased, roda 60% mais rápido e preserva mais de 95% do desempenho do Bert conforme medido no benchmark GLUE (General Language Understanding Evaluation).

[Hugging Face](https://huggingface.co/) (🤗) é o melhor recurso para transformers pré-treinados. Suas bibliotecas open-source simplificam o download, o fine-tuning e o uso de modelos de transformers como DeepSeek, BERT, Llama, T5, Qwen, GPT-2 e muito mais. E a melhor parte, você pode usá-los junto com TensorFlow, PyTorch ou Flax. Neste sistema, utilizo transformers 🤗 para usar o modelo `DistilBERT` para classificação de sentimento. Para a etapa do pré-processamento, usamos o tokenizador do DistilBERT `distilbert-base-uncased-finetuned-sst-2-english` pré-treinado, para isso inicializamos a classe DistilBertTokenizer e definidos o modelo pré-treinado desejado.
> No fine-tuning do modelo, no notebook `05_transformers_finetuning.ipynb`, falarei com mais detalhes sobre o transfer learning e o fine-tuning.

Tokenizando, aplicando o padding no corpus e retornando o corpus tokenizado como tensores pytorch utilizando o tokenizer `DistilBERT` pré-treinado. Definindo o tokenizer e plotando as representações vetoriais do corpus.

In [94]:
# Definindo o modelo pré-treinado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
# Definindo o tokenizer
comics_transformers = tokenizer(
    comics_data['description'].tolist(),
    return_tensors='pt',
    padding='max_length',
    truncation=True
)
# Acessando as representações vetoriais do corpus
transformers_tokens = comics_transformers['input_ids']
transformers_attention = comics_transformers['attention_mask']
transformers_tokens

tensor([[  101,  1999,  1996,  ...,     0,     0,     0],
        [  101,  2336,  1997,  ...,     0,     0,     0],
        [  101,  2336,  1997,  ...,     0,     0,     0],
        ...,
        [  101,  1037,  3181,  ...,     0,     0,     0],
        [  101,  1996,  2028,  ...,     0,     0,     0],
        [  101, 16228,  2023,  ...,     0,     0,     0]])

Selecionando os labels do dataset bruto e dividindo o tensor tokenizado entre os subsets de treinamento, de validação e de teste. Usamos o método de split `stratified sampling (amostragem estratificada)` para tentarmos manter a mesma proporção dos labels na divisão entre cada subset.

Dividindo o dataset e plotando o tamanho de cada um deles.

In [96]:
# Selecionando os labels do dataset bruto
labels = (comics_data['y']
          .map(lambda x: 1 if x == 'action' else 0)
          .to_numpy()
          .reshape(-1, 1))

# Dividindo entre treinamento e o subset da validação e teste
train_idx, subset_idx = next(split_train.split(transformers_tokens, labels))
# Dividindo entre validação e teste
val_idx, test_idx = next(split_test.split(transformers_tokens[subset_idx], labels[subset_idx]))
print(f'Tamanho do subset de treino: {len(train_idx)}\nTamanho do subset de validação: {len(val_idx)}\nTamanho do subset de teste: {len(test_idx)}')

Tamanho do subset de treino: 10156
Tamanho do subset de validação: 3385
Tamanho do subset de teste: 3386


Transformando os subsets de tensores pytorch tokenizados divididos pela `stratified sampling split` para o tipo `Dataset` em formato de dicionário, para executarmos o fine-tuning do transformers e plotando cada dataset.

In [98]:
train_dataset = tensors_to_dataset(transformers_tokens, transformers_attention, labels, train_idx)
val_dataset = tensors_to_dataset(transformers_tokens, transformers_attention, labels, val_idx)
test_dataset = tensors_to_dataset(transformers_tokens, transformers_attention, labels, test_idx)
print(f'Dataset de treino: {train_dataset}\nDataset de validação: {val_dataset}\nDataset de teste: {test_dataset}')

Dataset de treino: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10156
})
Dataset de validação: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3385
})
Dataset de teste: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3386
})


Carregando cada dataset pré-processado e seus metadados dentro de seu diretório específico dentro do diretório `../data/preprocessed/`.

In [65]:
train_dataset.save_to_disk('../data/preprocessed/train_dataset')
val_dataset.save_to_disk('../data/preprocessed/validation_dataset')
test_dataset.save_to_disk('../data/preprocessed/test_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/10156 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3385 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3386 [00:00<?, ? examples/s]